In [ ]:
import os
import subprocess
git_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode().strip()
%load_ext autoreload
%autoreload 2
os.chdir(git_root)
import ultralytics
print(os.path.dirname(ultralytics.__file__))
# https://docs.ultralytics.com/tasks/obb/#__tabbed_1_1
# https://docs.ultralytics.com/datasets/obb/ OBB Format
from ultralytics import YOLO
# Load a model
model = YOLO('yolov8n-obb.yaml')  # build a new model from YAML
model = YOLO('yolov8n-obb.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n-obb.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='DOTAv2.0-patches.yaml', epochs=20, imgsz=640, fraction=.01, batch=5)	 # data='dota8.yaml' multi_scale=True,mixup=1.0, 


In [ ]:
%%bash
# This script takes v1.0 and v2.0 original datasets and merges into ./examples/datasets/DOTA-v2.0 (DOTA format)
DATA_DIR=/home/vscode/datasets
echo $PWD
# Remove all previously exported files if they exist.
rm -rf $DATA_DIR/dota && mkdir -p $DATA_DIR/dota
# Remove DOTA-v2.0 dataset if exists.
rm -rf ./examples/datasets/DOTA-v2.0/ && mkdir -p ./examples/datasets/DOTA-v2.0/

rm -rf  $DATA_DIR/dota/v1.0 $DATA_DIR/dota/v2.0 && mkdir -p $DATA_DIR/dota/v1.0 $DATA_DIR/dota/v2.0
unzip -o -q -d $DATA_DIR/dota/v1.0 $DATA_DIR/dota-v1.0.zip
unzip -o -q -d $DATA_DIR/dota/v2.0 $DATA_DIR/dota-v2.0.zip
# Create directory structure for DOTA-v2.0 dataset according to ultralytics/yolov8
mkdir -p ./examples/datasets/DOTA-v2.0/{images,labels}/{train,val}

# Move files from v1.0 validation set to v2.0 validation set
for split in val train; do
    # Move data from v1.0,v2.0 to DOTA-v2.0/images
    find $DATA_DIR/dota/{v1.0,v2.0}/$split/images -name '*.png' -exec mv {} ./examples/datasets/DOTA-v2.0/images/$split \;
    # Move labels only from v2.0 to DOTA-v2.0/labels
    find $DATA_DIR/dota/v2.0/$split/labels -name '*.txt' -exec mv {} ./examples/datasets/DOTA-v2.0/labels/$split \;
done

In [ ]:
# Takes the original labels and converts them from DOTA to YOLO format

# Need to move them to an original directory to prepare for DOTA->YOLO conversion
!mv ./examples/datasets/DOTA-v2.0/labels/train ./examples/datasets/DOTA-v2.0/labels/train_original 
!mv ./examples/datasets/DOTA-v2.0/labels/val ./examples/datasets/DOTA-v2.0/labels/val_original
# Then the DOTA format needs to be converted to YOLO-OBB
from ultralytics.data.converter import convert_dota_to_yolo_obb
convert_dota_to_yolo_obb('./examples/datasets/DOTA-v2.0')


| Data Format                | Description                                                                 | New Column                          |
|---------------------------|-----------------------------------------------------------------------------|-------------------------------------|
| DOTA                      | `x1, y1, x2, y2, x3, y3, x4, y4, category, difficult`                        | OG DOTA format                      |
| YOLO-OBB                  | `class_index, x1, y1, x2, y2, x3, y3, x4, y4`                                | YOLO-OBB expected input format      |
| YOLO-OBB-internal         | `x,y,w,h, r`                                                               | YOLO-OBB internal format             |

### Data formatting in DOTA

There are three data formats:

- An image corresponds to a text file with the annotations i.e, **N** samples : `.png <--> .txt`

- Convert DOTA to `YOLO-OBB` using         

```python
from ultralytics.data.converter import convert_dota_to_yolo_obb

convert_dota_to_yolo_obb('path/to/DOTA')
```


### Baselines

Visit [here](https://github.com/dingjiansw101/AerialDetection/blob/master/MODEL_ZOO.md) for baselines

In [ ]:
# Creates a split where smalled images are cropped from large ones.

! rm -rf ./examples/datasets/DOTA-v2.0-split
from ultralytics.data.split_dota import split_trainval, split_test
from PIL import Image
Image.MAX_IMAGE_PIXELS = None # There is a security issue related to this.
# split train and val set, with labels.
split_trainval(
    data_root='./examples/datasets/DOTA-v2.0',
    save_dir='./examples/datasets/DOTA-v2.0-patches',
    rates=[0.5, 1.0, 1.5],    # multi-scale
    gap=500
)


In [ ]:
# NO test set right now.
# split test set, without labels.
if False:
    split_test(
        data_root='./examples/datasets/DOTA-v2.0',
        save_dir='./examples/datasets/DOTA-v2.0-split',
        rates=[0.5, 1.0, 1.5],    # multi-scale
        gap=500
    )

### Dev-env notes
- Remember pip install wandb
- Install ultralytics with `pip -e .`
- Create `./scripts/.env` file, put the wandb secret there
- If you dont want to log to wandb remember env `WANDB_MODE=offline`
- Remember to set `datasets_dirs: (git repo root)/examples/datasets` in ultralytics config at `~/.config/Ultralytics/settings.yaml`

### Data notes
- Remember that I add the classes 
```
        "airport": 16,
        "helipad": 17,
```

### General Notes

- Read A.Karpathy notes on training NNs [here](https://karpathy.github.io/2019/04/25/recipe/)
- Note on missing labels and how important they are [here](https://docs.ultralytics.com/yolov5/tutorials/tips_for_best_training_results/)

### Benchmarks
> OBB Results of pre-trained model from Ultralytics, evaluated on DOTAv1.0
```bash
$ export IMGSZ=640 && yolo obb val data=DOTAv2.0.yaml model=runs/obb/train-obb/weights/epoch99.pt batch=2 device=0 plots=True imgsz=$IMGSZ name=v2.0-inputsz$IMGSZ
```


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 
                   all        458      28853      0.689      0.427      0.463      0.281
                 plane        458       2531      0.782      0.614      0.647       0.41
                  ship        458       8960      0.733      0.478      0.555      0.285
          storage tank        458       2888      0.846      0.232      0.348      0.182
      baseball diamond        458        214      0.776      0.437      0.525      0.341
          tennis court        458        760      0.912      0.883      0.912      0.703
      basketball court        458        132      0.605      0.348       0.41       0.29
    ground track field        458        144      0.584      0.361      0.391      0.272
                harbor        458       2090      0.561      0.537      0.503      0.219
                bridge        458        464      0.591      0.125      0.146     0.0504
         large vehicle        458       4387      0.758      0.767      0.783      0.464
         small vehicle        458       5438      0.659      0.498      0.546       0.33
            helicopter        458         73       0.72       0.26      0.292      0.169
            roundabout        458        179      0.611      0.128       0.16     0.0962
     soccer ball field        458        153      0.551      0.392      0.369      0.226
         swimming pool        458        440       0.64      0.345      0.363      0.175

### Our results
> OBB Validation results after trainin on  DOTA-v2.0

| Parameter    | Value |
|--------------|-------|
| Train epochs | 100   |
| Wall time    | 16h   |
| Batch        | 90    |
| Time p epoch | 10m   |
| Data         | Dotav2.0-patches   |
| Resources    | 2xgpuv100(32GB) |
| Wandb link   | [here](https://wandb.ai/dimidagd_sl/YOLOv8/runs/0b2an0gd?workspace=user-dimidagd) |
```bash
$ IMGSZ=640 && yolo obb val data=DOTAv2.0.yaml model=runs/obb/train-obb/weights/epoch99.pt batch=2 device=0 plots=True imgsz=$IMGSZ name=v2.0-inputsz$IMGSZ
```
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)
                   all        590      80111      0.749      0.361      0.407      0.261
                 plane        590       2595      0.824      0.655      0.713      0.467
                  ship        590      13466      0.815      0.383      0.492      0.268
          storage tank        590       3136      0.871      0.227      0.327      0.188
      baseball diamond        590        226      0.829      0.409      0.502      0.346
          tennis court        590        777      0.905      0.875      0.919      0.748
      basketball court        590        147      0.659      0.347      0.416      0.313
    ground track field        590        165      0.569      0.439      0.432      0.325
                harbor        590       2249      0.717      0.601      0.625      0.281
                bridge        590        494      0.628     0.0911      0.134     0.0511
         large vehicle        590       5371       0.79      0.648      0.694      0.452
         small vehicle        590      50062      0.733      0.124      0.229      0.129
            helicopter        590         78      0.695      0.256      0.322      0.183
            roundabout        590        228      0.589     0.0921      0.111     0.0801
     soccer ball field        590        151      0.526      0.325      0.345      0.248
         swimming pool        590        851       0.74       0.25      0.334      0.176
       container crane        590         14          1          0          0          0
               airport        590         99      0.602      0.768      0.735      0.442
               helipad        590          2          1          0          0          0

### Plots

| Metric | Graph |
|--------|-------|
| F1 Score | ![F1 Score](../runs/obb/val6_DOTAv2-inputsz-640/F1_curve.png) |
| PR Curve | ![PR Curve](../runs/obb/val6_DOTAv2-inputsz-640/PR_curve.png) |
#### Test predictions
![Image](../runs/obb/val6_DOTAv2-inputsz-640/val_batch0_pred.jpg)
### Aerial photography tests
- Not great generalization on aerial imagery when the imgsz is lower than the image sz.
```bash
$ res=2000 && yolo obb predict model=runs/obb/train-obb/weights/epoch99.pt device=0  source='https://images.pexels.com/photos/681335/pexels-photo-681335.jpeg' conf=0.1 name=aerial_$res imgsz=$res project=static
```

| Input Resolution (imgsz=) | Image (native res. hw 3070x5464) |
|------------------|-------|
| 368x640px            | <img src="../runs/obb/aerial_subsubs/pexels-photo-681335.jpeg" alt="Image input res 620px" width="400"/> |
| 576x1024px           | <img src="../runs/obb/aerial_subs/pexels-photo-681335.jpeg" alt="Image input res 1024px" width="400"/> |
| 2272x4000px           | <img src="../runs/obb/aerial/pexels-photo-681335.jpeg" alt="Image input res 4000px" width="400"/> |